<center><h1> Data Selection</h1></center><br>


Here we will see, which dataset is more biased, or gives more mean squared error with same model. First I need to import libraries I need to check with which dataset my models perform better. To check I will use a technique called cross-validation. I will perform prediction on three machine learning algorithms. And then I import datasets I need. Also we need to shuffle our dataset. So far I made three different datasets. They will be shaffled, in script. Third dataset is mix of first and second dataset. Some values are dropped, some are imputed with another categorical values, and "Пробег" column is the only column subjected to predictive imputation, the reason I created the third dataset was in hope to maximally reduce the data.

In [3]:
import pandas as pd 
import pickle 
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [4]:
data_imputed = pd.read_excel("../data/interim/cars_imputed.xlsx")
data_dropped = pd.read_excel("../data/interim/cars_dropped.xlsx")
data_imputed = shuffle(data_imputed)
data_dropped = shuffle(data_dropped)

First I will divide the data into dataset and target variables and then I create predictor class instances and I will use cross-validation with 10 folds.

In [5]:
X_imp = data_imputed.iloc[:,:-1]
X_imp["Год выпуска"] = X_imp["Год выпуска"].astype(int).astype(str)
X_imp = pd.get_dummies(X_imp)
y_imp = data_imputed.iloc[:,-1]
X_drop = data_dropped.iloc[:,:-1]
X_drop["Год выпуска"] = X_drop["Год выпуска"].astype(int).astype(str)
X_drop = pd.get_dummies(X_drop)
y_drop = data_dropped.iloc[:,-1]

Cv argument means how many folds I want to use in cross-validation, more folds mean more unbiased error, for forest I use 5, because it's computationally more expensive.

In [6]:
#Linear Regression cv error
linear_imp = LinearRegression()
linear_drop = LinearRegression()
score_lr_imp = cross_val_score(linear_imp,X_imp,y_imp,scoring="mean_absolute_error",cv = 10) 
score_lr_drop = cross_val_score(linear_drop,X_drop,y_drop,scoring="mean_absolute_error",cv = 10) 
#KNN cv error
knn_imp = KNeighborsRegressor()
knn_drop = KNeighborsRegressor()
score_knn_imp = cross_val_score(knn_imp,X_imp,y_imp,scoring="mean_absolute_error",cv = 5) 
 #RandomForest cv error
forest_imp = RandomForestRegressor(n_estimators=30,n_jobs=1)
forest_drop = RandomForestRegressor(n_estimators=30,n_jobs=1)
score_forest_imp = cross_val_score(forest_imp,X_imp,y_imp,scoring="mean_absolute_error",cv = 5) 
score_forest_drop = cross_val_score(forest_drop,X_drop,y_drop,scoring="mean_absolute_error",cv = 5) 
score_knn_drop = cross_val_score(knn_drop,X_drop,y_drop,scoring="mean_absolute_error",cv = 5) 

C:\Users\AsusInside\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\AsusInside\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\AsusInside\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\AsusInside\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\U

In [8]:
print("MSE for imputed data for Random Forest:\t%.3f"%(score_forest_imp.mean()))
print("MSE for dropped data for Random Forest:\t%.3f"%(score_forest_drop.mean()))
print("MSE for imputed data for Linear Regression:%.3f"%(score_lr_imp.mean()))
print("MSE for dropped data for Linear Regression:%.3f"%(score_lr_drop.mean()))
print("MSE for imputed data for KNearestNeighbors:%.3f"%(score_knn_imp.mean()))
print("MSE for dropped data for KNearestNeighbors:%.3f"%(score_knn_drop.mean()))


MSE for imputed data for Random Forest:	-2755.851
MSE for dropped data for Random Forest:	-3714.164
MSE for imputed data for Linear Regression:-6081.842
MSE for dropped data for Linear Regression:-7920.117
MSE for imputed data for KNearestNeighbors:-6301.967
MSE for dropped data for KNearestNeighbors:-9594.430


We can clearly see that every model has better performance on imputed dataset, so we are going to work with imputed dataset.